In [1]:
import pandas as pd
import plotly.express as px
import sqlite3

In [2]:
cnx = sqlite3.connect('data/users-2023-08-17 22 29 22.468322.db')
messages_df = pd.read_sql_query("SELECT * FROM messages", cnx, parse_dates=['executed_at'])
users_df = pd.read_sql_query("SELECT * FROM users", cnx, parse_dates=['joined_at'])

In [12]:
messages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20206 entries, 0 to 20205
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              20206 non-null  int64         
 1   tg_message_id   20206 non-null  int64         
 2   used_tokens     20206 non-null  int64         
 3   has_image       20206 non-null  int64         
 4   executed_at     20206 non-null  datetime64[ns]
 5   user_id         20206 non-null  int64         
 6   personality     20206 non-null  object        
 7   history_size    20206 non-null  int64         
 8   query_tokens    1000 non-null   float64       
 9   time_taken      1000 non-null   float64       
 10  instant_buffer  20206 non-null  int64         
 11  model           20206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 1.9+ MB


In [13]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        228 non-null    int64         
 1   user_name      200 non-null    object        
 2   first_name     228 non-null    object        
 3   language_code  228 non-null    object        
 4   role           228 non-null    object        
 5   joined_at      228 non-null    datetime64[ns]
 6   ban            228 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 12.6+ KB


In [14]:
messages_df['user_id'].nunique(), users_df['user_id'].nunique()

(203, 228)

## Анализ пользователей

In [37]:
users_df['language_code'].value_counts() / users_df['language_code'].value_counts().sum() * 100

language_code
ru    82.456140
en    15.789474
es     1.315789
ko     0.438596
Name: count, dtype: float64

In [45]:
users_df['joined_date'] = users_df['joined_at'].dt.date
joined_by_date = users_df.groupby('joined_date').size().rename('Кол-во')

In [46]:
users_df['joined_week'] = users_df['joined_at'].dt.to_period('W').astype(str)
joined_by_week = users_df.groupby('joined_week').size().rename('Кол-во')

In [47]:
px.line(joined_by_date, title='Количество новых пользователей по дням')

In [48]:
px.line(joined_by_week, title='Количество новых пользователей по неделям')

## Анализ сообщений

In [3]:
messages_df['executed_date'] = messages_df['executed_at'].dt.date
executed_by_date = messages_df.groupby('executed_date').size().rename('Кол-во')

In [4]:
messages_df['executed_week'] = messages_df['executed_at'].dt.to_period('W').astype(str)
executed_by_week = messages_df.groupby('executed_week').size().rename('Кол-во')

In [51]:
px.line(executed_by_date, title='Количество сообщений по дням')

In [53]:
px.line(executed_by_week, title='Количество сообщений по неделям')

In [57]:
executed_by_date_users_count = messages_df.groupby('executed_date')['user_id'].nunique()
executed_by_week_users_count = messages_df.groupby('executed_week')['user_id'].nunique()

In [58]:
px.line(executed_by_date_users_count, title='Количество уникальных пользователей по дням')

In [59]:
px.line(executed_by_week_users_count, title='Количество уникальных пользователей по неделям')

In [75]:
usage_days = messages_df.groupby('user_id')['executed_date'].nunique()
fig = px.histogram(usage_days, title='Сколько дней пользователь общался с ботом (не подряд)', nbins=30)
fig.update_layout(bargap=0.2)
# fig.update_xaxes(type='category')

In [66]:
usage_days.describe()

count    203.000000
mean      11.714286
std       15.657761
min        1.000000
25%        2.000000
50%        5.000000
75%       14.000000
max       83.000000
Name: executed_date, dtype: float64

In [104]:
mean_usage_breaks_days = messages_df.groupby('user_id')['executed_date'].apply(lambda x: x - x.iloc[0]).apply(lambda x: x.days).groupby('user_id').agg('mean')
fig = px.histogram(mean_usage_breaks_days, title='Сколько дней пользователи делают перерыв в общении в среднем', nbins=30)
fig.update_layout(bargap=0.2)

In [107]:
mean_usage_breaks_days.describe()

count    203.000000
mean      17.321066
std       18.806062
min        0.000000
25%        0.645833
50%       10.352941
75%       28.240909
max       92.911111
Name: executed_date, dtype: float64

In [106]:
max_usage_breaks_days = messages_df.groupby('user_id')['executed_date'].apply(lambda x: x - x.iloc[0]).apply(lambda x: x.days).groupby('user_id').agg('max')
fig = px.histogram(max_usage_breaks_days, title='Сколько дней пользователи делают перерыв в общении максимально', nbins=30)
fig.update_layout(bargap=0.2)

In [108]:
max_usage_breaks_days.describe()

count    203.000000
mean      40.788177
std       36.563920
min        0.000000
25%        1.000000
50%       36.000000
75%       75.000000
max      100.000000
Name: executed_date, dtype: float64

In [26]:
fig = px.histogram(messages_df, x='query_tokens',title='Количество токенов в запросе пользователя', nbins=40)
fig.update_layout(bargap=0.2)

In [7]:
messages_df['query_tokens'].describe()

count    1000.000000
mean      107.233000
std       416.911165
min         1.000000
25%        13.000000
50%        25.000000
75%        62.000000
max      6872.000000
Name: query_tokens, dtype: float64

In [47]:
executed_by_date_query_tokens = messages_df.dropna().groupby('executed_date')['query_tokens'].mean()

In [48]:
px.line(executed_by_date_query_tokens, title='Среднее количество токенов в запросе пользователя по дням')

In [31]:
fig = px.histogram(messages_df, x='used_tokens', title='Количество токенов на полное сообщение (история + запрос + ответ)', nbins=40)
fig.update_layout(bargap=0.2)

In [32]:
messages_df['used_tokens'].describe()

count    20206.000000
mean      2282.623280
std       1232.866508
min         47.000000
25%       1227.000000
50%       2478.000000
75%       3226.000000
max      16032.000000
Name: used_tokens, dtype: float64

In [36]:
executed_by_date_used_tokens = messages_df.groupby('executed_date')['used_tokens'].sum()

In [39]:
px.line(executed_by_date_used_tokens, title='Суммарное количество токенов на полные сообщения по дням')

In [29]:
fig = px.histogram(messages_df, x='time_taken',title='Время ответа на запрос пользователя (мс)', nbins=40)
fig.update_layout(bargap=0.2)

In [16]:
messages_df['time_taken'].describe()

count      1000.000000
mean      17365.719000
std       11957.627441
min         303.000000
25%        9617.000000
50%       16433.500000
75%       22985.750000
max      142744.000000
Name: time_taken, dtype: float64

In [44]:
executed_by_date_time_taken = messages_df.dropna().groupby('executed_date')['time_taken'].mean()

In [45]:
px.line(executed_by_date_time_taken, title='Среднее время ответа на сообщение по дням')

In [25]:
messages_df[['used_tokens', 'time_taken']].corr()

,used_tokens,time_taken
used_tokens,1.000000,0.327981
time_taken,0.327981,1.000000


In [23]:
messages_df[['query_tokens', 'time_taken']].corr()

,query_tokens,time_taken
query_tokens,1.000000,0.055878
time_taken,0.055878,1.000000


In [54]:
messages_df['executed_day_from_begin'] = (messages_df['executed_date'] - messages_df['executed_date'].iloc[0]).apply(lambda x: x.days)
messages_df[['executed_day_from_begin', 'time_taken']].corr()

,executed_day_from_begin,time_taken
executed_day_from_begin,1.00000,0.18578
time_taken,0.18578,1.00000


In [21]:
messages_df[['history_size', 'time_taken']].corr()

,history_size,time_taken
history_size,1.000000,0.095482
time_taken,0.095482,1.000000


In [28]:
messages_df['history_size'].value_counts() / messages_df['history_size'].value_counts().sum() * 100

history_size
12    54.592695
1     15.361774
3      7.453232
5      5.879442
7      4.914382
9      4.157181
11     3.592992
14     2.162724
16     0.737405
13     0.351381
18     0.341483
20     0.257349
15     0.133624
17     0.059388
19     0.004949
Name: count, dtype: float64

In [15]:
messages_df['personality'].value_counts() / messages_df['personality'].value_counts().sum() * 100

personality
general            72.211224
programming        20.261309
custom              3.038701
summarizer          1.484708
joker               1.410472
psychotherapist     0.984856
en_edu              0.559240
chinese_edu         0.024745
music               0.024745
Name: count, dtype: float64